# Drugs for Treatments to OMOP

For the mapping in: https://docs.google.com/spreadsheets/d/1Vw1Dr2K4oG__cDQTutGaJhZvGUvQTLwc4qWreP6qMSs/edit?gid=634754097#gid=634754097 we just have the drug concept id. We assume a drug expossure can not exist with no treatment, and with this we will simply take this:

![First image](images/DrugsForTreatmentsIDEA1.png)

With episode_event_field_concept_id=1147070 we can do what is showed in the picture above.
